In [54]:
# === Simple average ensemble (5 models) ===
val_avg = (val_rf + val_hgb + val_xgb + val_lgbm + val_cat) / 5
print("RMSLE Avg Ensemble:", rmsle(y_val, val_avg))

# === Weighted average (adjust weights based on model performance if known)
val_weighted = (
    0.3 * val_xgb +
    0.25 * val_rf +
    0.25 * val_cat +
    0.1 * val_lgbm +
    0.1 * val_hgb
)
print("RMSLE Weighted Ensemble:", rmsle(y_val, val_weighted))

RMSLE Avg Ensemble: 0.060124599054829815
RMSLE Weighted Ensemble: 0.05992933242310699


In [55]:
# === Stacking: Use base model predictions as meta features
train_meta = np.column_stack((val_rf, val_hgb, val_xgb, val_lgbm, val_cat))
meta_model = Ridge(alpha=1.0, random_state=42)

# Train meta-model
meta_model.fit(train_meta, y_val)

# === Predict on test set with each base model
test_rf = rf.predict(X_test)
test_hgb = hgb.predict(X_test)
test_xgb = xgb.predict(X_test)
test_lgbm = lgbm.predict(X_test)
test_cat = cat.predict(X_test)

# === Meta test input and final stacked prediction
test_meta = np.column_stack((test_rf, test_hgb, test_xgb, test_lgbm, test_cat))
final_pred = meta_model.predict(test_meta)

In [56]:
test_cat = cat.predict(X_test)
train_meta = np.column_stack((val_rf, val_hgb, val_xgb, val_lgbm, val_cat))
test_meta = np.column_stack((test_rf, test_hgb, test_xgb, test_lgbm, test_cat))

meta_model = Ridge(alpha=1.0, random_state=42)
meta_model.fit(train_meta, y_val)

final_pred = meta_model.predict(test_meta)
final_val_pred = meta_model.predict(train_meta)
print("RMSLE (Stacked Ensemble with Tuned CatBoost):", rmsle(y_val, final_val_pred))


RMSLE (Stacked Ensemble with Tuned CatBoost): 0.06004187644932833


In [58]:
alphas = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
results = []

print("🔁 Tuning Ridge alpha...\n")

for alpha in alphas:
    ridge = Ridge(alpha=alpha, random_state=42)
    ridge.fit(train_meta, y_val)
    val_pred = ridge.predict(train_meta)
    score = rmsle(y_val, val_pred)
    results.append((alpha, score))
    print(f"Alpha: {alpha:<4} → RMSLE: {score:.6f}")

# Get best alpha
best_alpha, best_score = min(results, key=lambda x: x[1])
print(f"\n✅ Best alpha: {best_alpha} with RMSLE: {best_score:.6f}")


🔁 Tuning Ridge alpha...

Alpha: 0.01 → RMSLE: 0.060042
Alpha: 0.1  → RMSLE: 0.060042
Alpha: 0.5  → RMSLE: 0.060042
Alpha: 1.0  → RMSLE: 0.060042
Alpha: 2.0  → RMSLE: 0.060042
Alpha: 5.0  → RMSLE: 0.060042
Alpha: 10.0 → RMSLE: 0.060042

✅ Best alpha: 10.0 with RMSLE: 0.060042


In [59]:
meta_model = Ridge(alpha=best_alpha, random_state=42)
meta_model.fit(train_meta, y_val)
final_pred = meta_model.predict(test_meta)


In [63]:
from itertools import product

# Define possible weight values (must sum to 1)
weight_range = [0.1, 0.2, 0.3, 0.4, 0.5]

best_score = float('inf')
best_weights = None

print("🔁 Running grid search for weighted ensemble...\n")

for w in product(weight_range, repeat=5):
    if abs(sum(w) - 1.0) > 0.001:
        continue  # Skip if weights don't sum to ~1

    w_xgb, w_cat, w_rf, w_lgbm, w_hgb = w

    val_pred = (
        w_xgb * val_xgb +
        w_cat * val_cat +
        w_rf * val_rf +
        w_lgbm * val_lgbm +
        w_hgb * val_hgb
    )

    score = rmsle(y_val, val_pred)

    if score < best_score:
        best_score = score
        best_weights = w

        print(f"✅ New best RMSLE: {score:.6f} with weights: XGB={w_xgb}, CAT={w_cat}, RF={w_rf}, LGBM={w_lgbm}, HGB={w_hgb}")

print(f"\n🎯 Best weights found: {best_weights} → RMSLE: {best_score:.6f}")


🔁 Running grid search for weighted ensemble...

✅ New best RMSLE: 0.061352 with weights: XGB=0.1, CAT=0.1, RF=0.1, LGBM=0.2, HGB=0.5
✅ New best RMSLE: 0.061016 with weights: XGB=0.1, CAT=0.1, RF=0.1, LGBM=0.3, HGB=0.4
✅ New best RMSLE: 0.060783 with weights: XGB=0.1, CAT=0.1, RF=0.1, LGBM=0.4, HGB=0.3
✅ New best RMSLE: 0.060656 with weights: XGB=0.1, CAT=0.1, RF=0.1, LGBM=0.5, HGB=0.2
✅ New best RMSLE: 0.060516 with weights: XGB=0.1, CAT=0.1, RF=0.2, LGBM=0.3, HGB=0.3
✅ New best RMSLE: 0.060366 with weights: XGB=0.1, CAT=0.1, RF=0.2, LGBM=0.4, HGB=0.2
✅ New best RMSLE: 0.060323 with weights: XGB=0.1, CAT=0.1, RF=0.2, LGBM=0.5, HGB=0.1
✅ New best RMSLE: 0.060190 with weights: XGB=0.1, CAT=0.1, RF=0.3, LGBM=0.3, HGB=0.2
✅ New best RMSLE: 0.060124 with weights: XGB=0.1, CAT=0.1, RF=0.3, LGBM=0.4, HGB=0.1
✅ New best RMSLE: 0.060038 with weights: XGB=0.1, CAT=0.1, RF=0.4, LGBM=0.3, HGB=0.1
✅ New best RMSLE: 0.059983 with weights: XGB=0.1, CAT=0.2, RF=0.3, LGBM=0.3, HGB=0.1
✅ New best RMSLE:

In [65]:
# === Define Optuna objective ===
def weight_objective(trial):
    # Suggest weights for each model between 0 and 1
    w_xgb = trial.suggest_float("w_xgb", 0, 1)
    w_cat = trial.suggest_float("w_cat", 0, 1)
    w_rf = trial.suggest_float("w_rf", 0, 1)
    w_lgbm = trial.suggest_float("w_lgbm", 0, 1)
    w_hgb = trial.suggest_float("w_hgb", 0, 1)

    # Normalize weights to sum to 1
    total = w_xgb + w_cat + w_rf + w_lgbm + w_hgb
    w_xgb /= total
    w_cat /= total
    w_rf /= total
    w_lgbm /= total
    w_hgb /= total

    # Create blended prediction
    val_pred = (
        w_xgb * val_xgb +
        w_cat * val_cat +
        w_rf * val_rf +
        w_lgbm * val_lgbm +
        w_hgb * val_hgb
    )

    # Calculate RMSLE
    return rmsle(y_val, val_pred)

# === Run Optuna ===
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize")
study.optimize(weight_objective, n_trials=200)

# === Get best weights ===
best_weights = study.best_params
total = sum(best_weights.values())
normalized_weights = {k: v / total for k, v in best_weights.items()}
print("\n🎯 Best normalized weights found:", normalized_weights)
print("📉 Best RMSLE:", study.best_value)

# === Apply weights to test set ===
test_weighted = (
    normalized_weights['w_xgb'] * test_xgb +
    normalized_weights['w_cat'] * test_cat +
    normalized_weights['w_rf'] * test_rf +
    normalized_weights['w_lgbm'] * test_lgbm +
    normalized_weights['w_hgb'] * test_hgb
)

# === Save submission ===
sample['Calories'] = test_weighted
sample.to_csv('../outputs/submission_ensemble_optuna_v11.csv', index=False)
print("✅ Submission saved as '../outputs/submission_ensemble_optuna_v11.csv'")



🎯 Best normalized weights found: {'w_xgb': 0.17485098447315206, 'w_cat': 0.3358348494844964, 'w_rf': 0.3210744149301142, 'w_lgbm': 0.16696494966768965, 'w_hgb': 0.0012748014445478604}
📉 Best RMSLE: 0.05983711873380113
✅ Submission saved as '../outputs/submission_ensemble_optuna_v11.csv'


In [67]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
import numpy as np

# --- Base models with best Optuna params ---
# Assumes you've already saved: best_xgb_params, best_cat_params, etc.
# Load best params from previously tuned Optuna studies
xgb_params = xgb_study.best_params
cat_params = cat_study.best_params
rf_params = rf_study.best_params
lgbm_params = lgbm_study.best_params
hgb_params = hgb_study.best_params


# === Prepare OOF containers ===
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_preds = {
    'xgb': np.zeros(len(X)),
    'cat': np.zeros(len(X)),
    'rf': np.zeros(len(X)),
    'lgbm': np.zeros(len(X)),
    'hgb': np.zeros(len(X))
}
test_preds = {
    'xgb': np.zeros(len(X_test)),
    'cat': np.zeros(len(X_test)),
    'rf': np.zeros(len(X_test)),
    'lgbm': np.zeros(len(X_test)),
    'hgb': np.zeros(len(X_test))
}

# === Loop through folds ===
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"🔁 Fold {fold+1}/{n_folds}")
    X_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]
    X_val_fold = X.iloc[val_idx]

    # --- Train base models ---
    xgb = XGBRegressor(**xgb_params, random_state=42, verbosity=0)
    cat = CatBoostRegressor(**cat_params, random_state=42, verbose=0)
    rf = RandomForestRegressor(**rf_params, random_state=42, n_jobs=-1)
    lgbm = LGBMRegressor(**lgbm_params, random_state=42)
    hgb = HistGradientBoostingRegressor(**hgb_params, random_state=42)

    xgb.fit(X_tr, y.iloc[train_idx])
    cat.fit(X_tr, y.iloc[train_idx])
    rf.fit(X_tr, y.iloc[train_idx])
    lgbm.fit(X_tr, y.iloc[train_idx])
    hgb.fit(X_tr, y.iloc[train_idx])

    # --- OOF predictions ---
    oof_preds['xgb'][val_idx] = xgb.predict(X_val_fold)
    oof_preds['cat'][val_idx] = cat.predict(X_val_fold)
    oof_preds['rf'][val_idx] = rf.predict(X_val_fold)
    oof_preds['lgbm'][val_idx] = lgbm.predict(X_val_fold)
    oof_preds['hgb'][val_idx] = hgb.predict(X_val_fold)

    # --- Accumulate test set predictions (for final stacking) ---
    test_preds['xgb'] += xgb.predict(X_test) / n_folds
    test_preds['cat'] += cat.predict(X_test) / n_folds
    test_preds['rf']  += rf.predict(X_test) / n_folds
    test_preds['lgbm'] += lgbm.predict(X_test) / n_folds
    test_preds['hgb'] += hgb.predict(X_test) / n_folds

# === Stack OOF predictions for meta-model training ===
train_meta = np.column_stack((
    oof_preds['xgb'],
    oof_preds['cat'],
    oof_preds['rf'],
    oof_preds['lgbm'],
    oof_preds['hgb']
))

test_meta = np.column_stack((
    test_preds['xgb'],
    test_preds['cat'],
    test_preds['rf'],
    test_preds['lgbm'],
    test_preds['hgb']
))

# === Train Ridge meta-model ===
meta_model = Ridge(alpha=1.0, random_state=42)
meta_model.fit(train_meta, y)

# === Final stacked prediction ===
final_pred = meta_model.predict(test_meta)

# === Save submission ===
sample['Calories'] = final_pred
sample.to_csv('../outputs/submission_oof_stack_ridge_v1.csv', index=False)
print("✅ OOF stacking submission saved as '../outputs/submission_oof_stack_ridge_v1.csv'")


🔁 Fold 1/5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021688 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1670
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 16
[LightGBM] [Info] Start training from score 88.298465
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
🔁 Fold 2/5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017647 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1673
[LightGBM] [Info] Number of data points in the train set: 60000

In [60]:
# Save submission
sample['Calories'] = final_pred
sample.to_csv('../outputs/submission_ensemble_tuned_v9.csv', index=False)
print("✅ Submission file saved as '../outputs/submission_ensemble_tuned_v9.csv'")

✅ Submission file saved as '../outputs/submission_ensemble_tuned_v9.csv'
